In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import keras

In [ ]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
items.head()

In [ ]:
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')

In [ ]:
item_categories.head()

In [ ]:
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
shops.head()

In [ ]:
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
Id = test['ID']
test.head()

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
train['date'] = pd.to_datetime(train['date'])

In [ ]:
train.isna().sum()

In [ ]:
train['date'] = train['date'].apply(lambda x: x.strftime("%Y-%m"))

In [ ]:
train.head().sort_values(by = 'date')

In [ ]:
train.drop(['date_block_num','item_price'] , axis =1, inplace= True)

In [ ]:
train.head().sort_values(by = 'date')

In [ ]:
df = train.groupby(['date','shop_id','item_id']).sum()
df

In [ ]:
df = train.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df.head()

In [ ]:
test_df = pd.merge(test, df, on=['shop_id','item_id'], how='left')
test_df.drop(['ID', '2013-01'], axis=1, inplace=True)
test_df = test_df.fillna(0)

In [ ]:
test_df.head()

### Model

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

In [ ]:
Y = df['2015-10'].values
X = df.drop(['2015-10'], axis = 1)
test_full = test_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, lr.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, lr.predict(X_test)))
print('Train set score:', lr.score(X_train,y_train))

In [ ]:
lso = Lasso()
lso.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, lso.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, lso.predict(X_test)))
print('Train set score:', lso.score(X_train,y_train))

In [ ]:
ridge = Ridge()
ridge.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, ridge.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, ridge.predict(X_test)))
print('Train set score:', ridge.score(X_train,y_train))

In [ ]:
rndm_forst = RandomForestRegressor(n_estimators = 100)
rndm_forst.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, rndm_forst.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, rndm_forst.predict(X_test)))
print('Train set score:', rndm_forst.score(X_train,y_train))

In [ ]:
grad_boost = GradientBoostingRegressor(learning_rate = 0.1, n_estimators = 100, random_state = 42)
grad_boost.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, grad_boost.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, grad_boost.predict(X_test)))
print('Train set score:', grad_boost.score(X_train,y_train))

In [ ]:
prediction = grad_boost.predict(test_full)

In [ ]:
prediction = list(map(round, prediction))

In [ ]:
sample_sub = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(sample_sub.shape)
sample_sub.head()

In [ ]:
sample_sub['item_cnt_month'] = prediction
sample_sub.to_csv('prediction.csv', index=False)
sample_sub.head()